# TP1: Agrupamiento de imágenes

In [5]:
import numpy as np
from PIL import Image

Data Mining en Ciencia y Tecnología

## 1. Introducción
El procesamiento de imágenes resulta desafiante por su alta dimensionalidad. La estructura de una imagen digital consiste en una matriz de NxM, en donde la subunidad constituyente de la matriz es un pixel que codifica información para un color particular. Cada pixel representa la intesidad de luz en ese punto, que generalmente varía entre [0,255], lo que es equivalente a 8 bits.
Para representar imágenes a colores, se utiliza un modelo de percepción humana, en donde el color resulta a través de un sistema aditivo. El modelo se basa en la teoría de los componentes primarios del color que son Rojo, Verde y Azul (RGB Red, Green and Blue, por sus siglas en inglés). Por consiguiente, para representar digitalmente una imagen color, se necesitan 3 matrices de NxM . Una para el Rojo, otra para el Verde y otra para el Azul.

## 2. Objetivos
Familiarizarse con el procesamiento de imágenes. Para ello, se proponen diferentes ma- nipulaciones que permitirán preparar el dataset para la detección y exploración de agrupa- mientos naturales.

## 3. Estructura de los datos:
A partir del siguiente link, se obtendrán las imágenes a color de 210 flores pertenecientes a 10 especies diferentes. Cada imagen consiste en un archivo .PNG de 128 pixeles de ancho por 128 pixeles de profundidad (128x128x3). Adicionlamente, se encuentra el archivo .CSV con las etiquetas (labels) que corresponden a la especie de cada imagen.

## 4. Preprocesamiento de los datos
- Cargar el dataset y sus respectivas etiquetas. Es importante asegurarse que las imágenes sean comparables en color, valor, rango y tamaño.
- Explorar y graficar los subconjuntos de imágenes que representan flores de la misma especie.

In [12]:
! kaggle datasets download -d olgabelitskaya/flower-color-images -p ../../data/raw
! unzip ../../data/raw/flower-color-images.zip -d ../../data/raw

100%|█████████████████████████████████████▉| 50.0M/50.1M [00:04<00:00, 8.27MB/s]
100%|██████████████████████████████████████| 50.1M/50.1M [00:04<00:00, 10.6MB/s]
Archive:  ../../data/raw/flower-color-images.zip
  inflating: ../../data/raw/FlowerColorImages.h5  
  inflating: ../../data/raw/flower_images/flower_images/0001.png  
  inflating: ../../data/raw/flower_images/flower_images/0002.png  
  inflating: ../../data/raw/flower_images/flower_images/0003.png  
  inflating: ../../data/raw/flower_images/flower_images/0004.png  
  inflating: ../../data/raw/flower_images/flower_images/0005.png  
  inflating: ../../data/raw/flower_images/flower_images/0006.png  
  inflating: ../../data/raw/flower_images/flower_images/0007.png  
  inflating: ../../data/raw/flower_images/flower_images/0008.png  
  inflating: ../../data/raw/flower_images/flower_images/0009.png  
  inflating: ../../data/raw/flower_images/flower_images/0010.png  
  inflating: ../../data/raw/flower_images/flower_images/0011.png  
 

## 5. Manipulación de datos
- Cambiar la intensidad de una de las imágenes en escala de grises, transformarla en una imagen con mucho y otra con poco brillo.
- Convertir una de las imágenes a blanco y negro (binario). ¿Es la única manera? Si existen otras transformaciones mostrar más de una conversión.
- Recortar una parte significativa de la imagen, quedándose sólo con el círculo central de la misma.
- Generar dos imágenes random: una imagen mezclando los pixels y otra mezclando partes de diferentes imágenes.
- Aplicar dos tipos diferentes de filtros sobre una imagen, explique en qué casos conviene usar cada uno.
- Calcular imagen promedio global y el promedio entre las distintas especies. ¿Se pueden distinguir los promedios? ¿Cómo quedan los promedios si consideran las imágenes en blanco y negro?

In [ ]:
import pandas as pd